In [ ]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# name of the cluster
compute_cluster_name = "alpha"

try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print("Found existing cluster, please use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        "--C": uniform(0.1, 1.0),
        "--max_iter": choice(25, 50, 100, 150)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=".", compute_target=compute_cluster, entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     policy = policy,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdr = exp.submit(hyperdrive_config)
RunDetails(hdr).show()
hdr.wait_for_completion(show_output=True)

assert(hdr.get_status() == "Completed")

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run_hdr = hdr.get_best_run_by_primary_metric()
best_run_metrics_hdr = best_run_hdr.get_metrics()
best_params_hdr = best_run_hdr.get_details()['runDefinition']['arguments']

#print(best_params_hdr)

print("\n")
print('Best run ID: ', best_run_hdr.id)
print('Metrics: ', best_run_hdr.get_metrics())
print("\n")

print(best_run_hdr.get_file_names())

# Register model
best_run_hdr.upload_file('outputs/best_model_hyperdrive.joblib', 'outputs/best_model_hyperdrive.joblib')
model = best_run_hdr.register_model(model_name='best_model_hyperdrive', model_path='outputs/best_model_hyperdrive.joblib')
joblib.dump(best_params_hdr, filename='outputs/best_model_hyperdrive.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=data_path)

In [ ]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from sklearn.model_selection import train_test_split
from azureml.core import Dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Train data with target
train_data = pd.concat([X_train, y_train], axis=1)
train_data.to_csv("training/train_data.csv", index=False)

# Store train data
data_store = ws.get_default_datastore()
data_store.upload(src_dir="./training", target_path="udacity-project", overwrite=True, show_progress=True)

# Reload train data
train_data = Dataset.Tabular.from_delimited_files(path=data_store.path("udacity-project/train_data.csv"))

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name='y',
    n_cross_validations=4,
    compute_target=compute_cluster)

In [ ]:
# Submit your automl run

automl = exp.submit(automl_config)
RunDetails(automl).show()
automl.wait_for_completion(show_output=True)

assert(automl.get_status() == "Completed")

In [ ]:
# Retrieve and save your best automl model.

best_run_automl, best_model_automl = automl.get_output()
print(best_run_automl)
print("\n")
print(best_model_automl)

# Register model
best_run_automl.upload_file('outputs/best_model_automl.joblib', 'outputs/best_model_automl.joblib')
model = best_run_automl.register_model(model_name='best_model_automl', model_path='outputs/best_model_automl.joblib')
joblib.dump(best_model_automl, filename='outputs/best_model_automl.joblib')

In [ ]:
# Cleanup cluster
compute_cluster.delete()